Notes and Todos:
* put results handling in a function [done]


In [ ]:
# IJulia.qtconsole();

In [1]:
import Pkg
Pkg.activate(".")
# Pkg.instantiate()

"C:\\Users\\Robert\\Promotion\\Workshop\\02_cc_dlmp_varaware\\01_case_study_cc_dlmp\\Project.toml"

In [2]:
using Logging
using DataFrames, CSV, LinearAlgebra
using JuMP
# using Gurobi, Ipopt
using Mosek, MosekTools

In [3]:
include("src/input.jl");
include("src/model_definition.jl")
include("src/output.jl")

results_to_df (generic function with 1 method)

In [4]:
datadir = ("data/feeder_data/basecase_noneg/")
# datadir = ("data/feeder_data/feeder141/")
# datadir = ("data/feeder_data/feeder141_lv/")

feeder = load_feeder(datadir);

>>>>> Reading feeder data from data/feeder_data/basecase_noneg/


In [5]:
relative_std = 0.1
loads = [b.d_P for b in feeder.buses]
std_vec = loads .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

casesettings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945 ,
    "z_v" => 1.945,
    "toggle_volt_cc" => true,
    "toggle_gen_cc" => true,
    "vfac" => 0,
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
)

Dict{String,Any} with 10 entries:
  "var_vec"        => [0.0, 0.00629801, 0.0, 4.0401e-6, 2.9929e-6, 8.4681e-6, 4…
  "toggle_volt_cc" => true
  "qcfac"          => 1
  "Ψ"              => 0
  "Σ"              => [0.0 0.0 … 0.0 0.0; 0.0 0.00629801 … 0.0 0.0; … ; 0.0 0.0…
  "toggle_gen_cc"  => true
  "output_level"   => 0
  "z_g"            => 1.945
  "z_v"            => 1.945
  "vfac"           => 0

In [6]:
m, meta = build_model(feeder, casesettings)

(A JuMP Model
Minimization problem with:
Variables: 121
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 47 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 99 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 33 constraints
`Array{GenericAffExpr{Float64,VariableRef},1}`-in-`MathOptInterface.SecondOrderCone`: 15 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek
Names registered in the model: fp, fq, gp, gp0_minus, gp0_plus, gq, linear_cost, r, t, v, variance_penalty, α, β, γ, δm, δp, ζ, η, θm, θp, λ, μm, μp, π, ρ, Dict{String,Any}("idx_to_bus"=>Dict{Any,Any}(2=>3,11=>12,7=>8,9=>10,10=>11,8=>9,6=>7,4=>5,3=>4,5=>6…),"bus_to_idx"=>Dict{Any,Any}(2=>1,11=>10,7=>6,9=>8,10=>9,8=>7,6=>5,4=>3,3=>2,5=>4…),"any

In [7]:
optimize!(m)
status = termination_status(m)

MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(87) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(93) of matrix 'A'.


OPTIMAL::TerminationStatusCode = 1

In [8]:
results_df = results_to_df(m, meta, feeder)

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any
1,28.6173,1,-1.41085e-7,0.919899,1.0,0.765167,-20.9929,-1.59254e-8,-8.18012,0,0,0,0,0,0
2,0.0,2,0,0,0.902044,0,-20.9929,-5.01859e-5,0.0,-7.89672e-9,0,0,-4.37564e-7,-0.000276133,0.000567509
3,0.0,3,0,0,0.957801,0,-20.9929,-3.3343e-5,0.0,-4.41779e-7,0,0,-7.39e-7,-3.49148e-6,0.0178157
4,0.0,4,0,0,1.03919,0,-20.9929,-5.77372e-6,0.0,-1.11578e-6,0,0,-4.19102e-6,-5.79349e-7,0.022235
5,0.0,5,0,0,1.04772,0,-20.9929,-3.96773e-6,0.0,-1.16917e-6,0,0,-5.20429e-6,-4.84486e-7,0.0196763
6,0.0,6,0,0,1.05584,0,-20.9929,-2.53523e-6,0.0,-1.21501e-6,0,0,-6.52909e-6,-4.69156e-7,0.0169503
7,0.0,7,0.902837,-0.240921,1.0797,0.117553,-20.9929,-5.86274e-7,0.0,-1.32771e-6,-8.73689,-2.34247e-7,-2.33063e-5,-4.83834e-7,0.00844098
8,0.0,8,0,0,1.03514,0,-20.9929,-1.4242e-6,0.0,-1.20397e-6,0,0,-4.211e-6,-7.67079e-7,0.0223299
9,0.0,9,0,0,1.04518,0,-20.9929,-1.9113e-6,0.0,-1.19566e-6,0,0,-5.58678e-6,-6.73484e-7,0.0196083


In [ ]:
saveres = results_df

In [ ]:
CSV.write("figures/141res_lv.csv", saveres)

In [ ]:
mu = results_df[:mu_plus] .+ results_df[:mu_minus]
mu = mu[2:end]

b = 1 ./ diag(C)
b = [(x == Inf ? 0 : x) for x in b]

g3 = 2*s*z*e'*R'*mu/sum(b)

g1 = s^2/sum(b) 

delta = results_df[:delta_plus] .+ results_df[:delta_minus]
detlta = delta[2:end]
g2 = sum(delta.*b)/sum(b)

g = -g1 - g2 + g3


In [ ]:
saveres = results_df

In [ ]:
CSV.write("figures/results_uncert.csv", results_df)

In [ ]:
shadow_price(m[:δp][12])

In [ ]:
m.dictList